## 1. Examine both Open Meteo and Tomorrow and decide which one you'd prefer to use. What drove your decision?

### Open Meteo seems slightly more... approachable? But Tomorrow has all those companies using it, huh...

## 2. What is the URL to the documentation? (You don't use code for this one)

### https://open-meteo.com/en/docs

## 3. Make a request for the current weather where you are born, or somewhere you've lived.

In [37]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 38.6273,
	"longitude": -90.1979,
	"current": ["temperature_2m", "apparent_temperature", "relative_humidity_2m"],
	"timezone": "Europe/London",
	"forecast_days": 1
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")

# Current values. The order of variables needs to be the same as requested.
current = response.Current()
current_temperature_2m = current.Variables(0).Value()
current_relative_humidity_2m = current.Variables(1).Value()

print(f"Current temperature in St. Louis: {current_temperature_2m:.2f}")
print(f"Current relative humidity in St. Louis: {current_relative_humidity_2m:.2f}")

Coordinates 38.62271499633789°N -90.21289825439453°E
Current temperature in St. Louis: 22.49
Current relative humidity in St. Louis: 24.03


## 4. Print out the country this location is in.

#### I'm not actually sure I can retrieve that? Like you search by latitude and longitude, but the response doesn't seem to retrieve that...

## 5. Print out the difference between the current temperature and how warm it feels. Use "It feels ___ degrees colder" or "It feels ___ degrees warmer," not negative numbers.


In [38]:
current_temp = current.Variables(0).Value()
feelslike_temp = current.Variables(1).Value()

temp_diff = round((current_temp - feelslike_temp), 2)

if temp_diff > 0:
    print(f"It feels {temp_diff} degrees Fahrenheit colder")
else:
    print(f"It feels {abs(temp_diff)} degrees Fahrenheit warmer")

It feels 1.54 degrees Fahrenheit warmer


## 6. What's the current temperature at Heathrow International Airport? Use the airport's IATA code to search.

#### First off - you can't use IATA to search, only lat and longitude

In [30]:
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 51.4704,
	"longitude": -0.4586,
	"current": "temperature_2m"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]

# Current values. The order of variables needs to be the same as requested.
current = response.Current()
current_temperature_2m = current.Variables(0).Value()

print(f"Current temperature at Heathrow Airport: {current_temperature_2m:.2f}")

Current temperature at Heathrow Airport: 21.15


## 7. What URL would I use to request a 3-day forecast at Heathrow?

In [41]:
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 51.4704,
	"longitude": -0.4586,
	"daily": ["temperature_2m_max", "temperature_2m_min"],
	"timezone": "Europe/London",
	"forecast_days": 3
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["temperature_max"] = daily_temperature_2m_max
daily_data["temperature_min"] = daily_temperature_2m_min

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

Timezone difference to GMT+0 3600 s
                       date  temperature_max  temperature_min
0 2025-06-10 23:00:00+00:00        22.346001           12.546
1 2025-06-11 23:00:00+00:00        24.046000           13.896
2 2025-06-12 23:00:00+00:00        23.746000           15.046


## 8. Print the date of each of the 3 days you're getting a forecast for.

#### See above!

## 9. Print the maximum temperature of each of the days.

#### See above!

## 10. Print only the day with the highest maximum temperature.

In [56]:
max_temp_row = daily_dataframe.loc[daily_dataframe['temperature_max'].idxmax()]
print(f"Not sure how best to format this, but! {max_temp_row['date']}")

Not sure how best to format this, but! 2025-06-11 23:00:00+00:00


## 11. Did you find this easier or more difficult than using the weatherapi.com, and why? Which would you recommend to someone interesting in building a tool around weather information?


In [ ]:
#### I guess it really depends on how granular they want to go. I feel like they'd have to build some sort of layer for interpreting latitude and longitude, assuming that there's not a better way to look up locations with coordinates. I mostly used the docs for